# Previsão dos Hits do Spotify

##### Neste notebook é criado um algoritmo que com base nos arquivos fornecidos no kaggle faz uma previsão de qual será a próxima música popular no spotify

## Importação das bibliotecas utilizadas no código

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt

## Importação dos dados

In [3]:
#Importando dados
treino = pd.read_csv('train.csv')
teste = pd.read_csv('test.csv')

## Análise dos dados

In [ ]:
#Fazendo análise dos dados de treino
treino.head()

In [ ]:
#Fazendo análise dos dados de treino
teste.head()

In [ ]:
#Analisando informações gerais dos dados de treino
treino.info()

In [ ]:
#Verificando a existênia de dados nulos no conjunto de treino
treino.isnull().sum()       

In [ ]:
#Verificando a existênia de dados nulos no conjunto de teste
teste.isnull().sum()

## Limpeza e tratamento dos dados

### Tratamento dos valores nulos

In [ ]:
# Tratando valores nulos preenchendo com strings vazia para colunas categóricas
teste['artists'].fillna('', inplace=True)
teste['album_name'].fillna('', inplace=True)
teste['track_name'].fillna('', inplace=True)

### Codificação das variáveis categóricas

In [10]:
# Codificando variáveis categóricas explicit e mode no dataset treino e teste
treino['explicit'] = treino['explicit'].astype('category')
treino['mode'] = treino['mode'].astype('category')

teste['explicit'] = teste['explicit'].astype('category')
teste['mode'] = teste['mode'].astype('category')

#Convertendo explicit e mode para variáveis numéricas
treino['explicit'] = treino['explicit'].cat.codes
treino['mode'] = treino['mode'].cat.codes

teste['explicit'] = teste['explicit'].cat.codes
teste['mode'] = teste['mode'].cat.codes

### Confirmação do tratamento

In [ ]:
#Verificando a coficação no conjunto de treino
treino[['explicit', 'mode']].head()

In [ ]:
#Verificando a coficação no conjunto de teste
teste[['explicit', 'mode']].head()

### Normalização dos dados

In [13]:
#Aplicar escalonaento nos dados numéricos
scaler = StandardScaler()

colunas_numericas = ['duration_ms', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

treino[colunas_numericas] = scaler.fit_transform(treino[colunas_numericas])
teste[colunas_numericas] = scaler.transform(teste[colunas_numericas])


### Codificação das variáveis categóricas

In [14]:
# Codificando variáveis categóricas explicit e mode no dataset treino e teste
treino['track_genre'] = treino['track_genre'].astype('category').cat.codes
teste['track_genre'] = teste['track_genre'].astype('category').cat.codes


In [ ]:
#Verificando a coficação no conjunto de treino
treino[colunas_numericas + ['track_genre']].head()

In [ ]:
#Verificando a coficação no conjunto de teste
teste[colunas_numericas + ['track_genre']].head()

## Exploração dos dados

In [ ]:
#Observando a correlação das variáveis para formulação de hipóteses
corr_matrix = treino[colunas_numericas + ['popularity_target']].corr()

# Exibir um heatmap das correlações
plt.figure(figsize=(9, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlação das variáveis')
plt.show()


# Hipóteses

Primeira Hipótese: Gêneros influenciam na popularidade da música 

Justificativa: Existem certos gêneros que tendem a fazer mais sucesso

In [ ]:
genre_popularity = treino.groupby('track_genre')['popularity_target'].mean().sort_values(ascending=False)

# Selecionando os 20 gêneros de música mais populares
top_20_genres = genre_popularity.head(20)

# Plotando os 20 gêneros mais populares e suas popularidades
plt.figure(figsize=(10, 6))
top_20_genres.plot(kind='bar', color='orange')

# Adicionando título e legendas
plt.title('Top 20 gêneros mais populares', fontsize=14)
plt.xlabel('Gênero Musical', fontsize=12)
plt.ylabel('Popularidade Média', fontsize=12)

# Rotacionando eixo x
plt.xticks(rotation=45)

# Plotando gráfico
plt.tight_layout()
plt.show()

Esse gráfico mostra que existem gêneros mais populares e que músicas desse gêneros, portanto, possuem uma maior possibilidade de fazerem sucesso.

Segunda Hipótese: Músicas com maior volume fazem mais sucesso 

Justificativa: Músicas com maior volume geralmente são músicas mais voltadas para o público portanto tendem a fazer mais sucesso

In [ ]:
# Filtrando músicas populares (definindo como popular aquelas com popularidade maior que uma certa nota, por exemplo 1)
musicas_populares = treino[treino['popularity_target'] == 1]

# Criando um histograma para volume (loudness) das músicas populares
plt.figure(figsize=(8, 6))
plt.hist(musicas_populares['loudness'], bins=20, color='green', edgecolor='black')

# Adicionando título e rótulos
plt.title('Quantidade de Músicas Populares por Intervalo de Volume', fontsize=14)
plt.xlabel('Volume (dB)', fontsize=12)
plt.ylabel('Quantidade de Músicas', fontsize=12)

# Exibindo o gráfico
plt.tight_layout()
plt.show()

Esse gráfico demonstra que a maior parte das músicas populares são as de maior volume no gráfico

Terceira Hipótese: Músicas mais energéticas tendem a ser mais sucesso

Justificativa: Músicas mais energéticas são comumente produzidas para serem tocadas em festas o que as da uma maior chance de se tornarem populares

In [ ]:
# Filtrando músicas populares (definindo como populares aquelas com popularidade maior que uma certa nota, por exemplo, 1)
musicas_populares = treino[treino['popularity_target'] == 1]

# Criando um histograma para a energia das músicas populares
plt.figure(figsize=(8, 6))
plt.hist(musicas_populares['energy'], bins=20, color='orange', edgecolor='black')

# Adicionando título e rótulos
plt.title('Quantidade de Músicas Populares por Intervalo de Energia', fontsize=14)
plt.xlabel('Energia', fontsize=12)
plt.ylabel('Quantidade de Músicas', fontsize=12)

# Exibindo o gráfico
plt.tight_layout()
plt.show()

Esse gráfico demonstra que a maior parte das músicas populares são as de maior energia 

## Seleção de features

In [21]:
#Separando as features e o alvo
X = treino.drop(columns=['popularity_target', 'track_unique_id', 'track_id', 'artists', 'album_name', 'track_name'])
y= treino['popularity_target']

## Construção e Avaliação do Modelo

In [22]:
#instaciando o modelo
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

In [23]:
# Dividindo treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
# Convertendo os dados para o formato DMatrix do XGBoost
dados_treino = xgb.DMatrix(X_train, label=y_train)
dados_teste = xgb.DMatrix(X_val, label=y_val)

In [25]:
# Definindo os parâmetros do XGBoost
params = {
    'objective': 'multi:softmax',  # Multiclass classification
    'num_class': 3,  # Número de classes
    'max_depth': 15,  # Profundidade da árvore
    'eta': 0.3,  # Taxa de aprendizado
    'eval_metric': 'mlogloss'  # Métrica de avaliação
}

In [ ]:
# Treinando o modelo
model = xgb.train(params, dados_treino, num_boost_round=100)

# Fazendo previsões
previsoes = model.predict(dados_teste)

# Avaliando o modelo
accuracy = accuracy_score(y_val, previsoes)
print(f'Acurácia: {accuracy * 100:.2f}%')

In [ ]:
#Relatório de desempenho
print(classification_report(y_val, previsoes))

In [28]:
# Fazendo previsões com conjunto de teste
X_teste = teste.drop(columns=['track_unique_id', 'track_id', 'artists', 'album_name', 'track_name'])
# Convertendo os dados para o formato DMatrix do XGBoost
dteste = xgb.DMatrix(X_teste)

previsoes_teste = model.predict(dteste)

# Gerar arquivo CSV para submissão
submission = teste[['track_unique_id']].copy()
submission['popularity_target'] = previsoes_teste
submission.to_csv('submission.csv', index=False)

## Finetuning de Hiperparâmetros

In [ ]:
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None]
}

grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

print(f'Melhores parâmetros: {grid_search.best_params_}')

## Métricas do modelo

In [ ]:
# Previsões no conjunto de validação usando o melhor modelo encontrado
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_val)

# Calcular a acurácia
accuracy = accuracy_score(y_val, y_pred)
print(f'Acurácia com o modelo otimizado: {accuracy:.2f}')

# Exibir outras métricas de avaliação
print("Relatório de Classificação:\n", classification_report(y_val, y_pred))